In [ ]:
## RUN THIS CELL IF AND ONLY IF THIS IS OPEN IN GOOGLE COLAB
!git clone https://github.com/ries-lab/SimuFLUX.git
%cd SimuFLUX/python/examples

In [ ]:
%matplotlib inline

import sys
import os
from pathlib import Path

SCRIPT_DIR = Path(os.getcwd()).parent
sys.path.append(os.path.dirname(SCRIPT_DIR))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from python.fluorophores import FlStatic
from python.psfs import PsfVectorial
from python.estimators import est_donutLSQ1_2D
from python.estimators import est_qLSQiter2D
from python.estimators import est_qDirectFitBg1D
from python.estimators import est_qMLE1D
from python.estimators import est_GaussLSQ1_2D
from python.estimators import backgroundsubtractor
from python.simulators import Simulator

In [ ]:
psf_vec = PsfVectorial()
psf_vec.setpinhole(AU=1)

In [ ]:
fl = FlStatic()
fl.brightness = 10000  # very bright to look at bias
fl.pos = [0, 0, 0]
sim = Simulator(fluorophores=fl)

In [ ]:
numberOfLocalizations = 1000
L = 75
orbitpoints = 4
laserpower = 100
# xcoords = np.arange(0,L*0.75,2)
xcoords = np.arange(0,L+5,5)
probecenter = True
sim.definePattern("donut", psf_vec, 
                  phasemask="vortex", 
                  makepattern="orbitscan", 
                  orbitpoints=orbitpoints,
                  probecenter=probecenter,
                  orbitL=L,
                  laserpower=laserpower)

## No background, simple estimator

In [ ]:
ax1v = "pos" 
sim.defineComponent("estdonut", "estimator", est_donutLSQ1_2D, parameters=[sim.patterns["donut"].pos, L, 360], dim=(0,1))
seq=["donut", "estdonut"]
psf_vec.zerooffset = 0

fig = plt.figure()
ax = fig.add_subplot(121)
statout = sim.scan_fov(seq, xcoords, maxlocs=numberOfLocalizations, 
                       display=True, ax1=ax1v, clearfigure=True,
                       tag="simple est")
phottxt = f"photons. Simple: {statout.phot[0]}"

In [ ]:
# iterative
sim.defineComponent("estiter", "estimator", est_qLSQiter2D, parameters=[L,probecenter,20], dim=(0,1))
seq = ["donut", "estiter"]
statout = sim.scan_fov(seq, xcoords, maxlocs=numberOfLocalizations, 
                       display=True, ax1=ax1v, clearfigure=False, 
                       tag="iterative est")
phottxt += f", iter: {statout.phot[0]}"

In [ ]:
sim.defineComponent("estimator2p", "estimator", est_qDirectFitBg1D, parameters=[L,probecenter], dim=0)
seq = ["donut", "estimator2p"]
statout=sim.scan_fov(seq, xcoords, maxlocs=numberOfLocalizations, 
                     display=True, ax1=ax1v, clearfigure=False, 
                     tag="direct eq")
phottxt += f", direct: {statout.phot[0]}"

In [ ]:
sim.defineComponent("estimatorMLE", "estimator", est_qMLE1D, parameters=[L], dim=0)
seq = ["donut", "estimatorMLE"]
statout=sim.scan_fov(seq, xcoords, maxlocs=numberOfLocalizations, 
                     display=True, ax1=ax1v, clearfigure=False, 
                     tag="MLE")
phottxt += f", MLE: {statout.phot[0]}"

In [ ]:
LG = L*2
sim.definePattern("gauss", psf_vec, 
                  phasemask="flat", 
                  makepattern="orbitscan", 
                  orbitpoints=orbitpoints,
                  probecenter=probecenter,
                  orbitL=LG,
                  laserpower=laserpower)

sim.defineComponent("estimatorGauss", "estimator", est_GaussLSQ1_2D, parameters=[sim.patterns["gauss"].pos, LG, 110, probecenter], dim=0)
seq = ["donut", "estimatorGauss"]
statout=sim.scan_fov(seq, xcoords, maxlocs=numberOfLocalizations, 
                     display=True, ax1=ax1v, clearfigure=False, 
                     tag="Gauss")
phottxt += f", Gauss: {statout.phot[0]}"

In [ ]:
fig.suptitle(phottxt)
ax = plt.gca() 
if ax1v=="pos":
    ax.plot([0, L],[0, L],'k--')
ax.set_ylim([0, 65])
ax.set_xlim(right=L*0.75)

In [ ]:
fig.tight_layout()

## Explore impact of background on estimator

In [ ]:
# no background
fl.brightness = 2000
fl.pos = [0, 0, 0]
sim.background = 0
sim.defineComponent("estdonut", "estimator",est_qLSQiter2D, parameters=[L,probecenter], dim=(0,1))
seq = ["donut", "estdonut"]
psf_vec.zerooffset=0

ax = plt.subplot(122)
statout=sim.scan_fov(seq, xcoords, maxlocs=numberOfLocalizations, 
                     display=True, ax1=ax1v, tag="no bg")
photons = statout.phot[0]

In [ ]:
# background, 
sim.background = fl.brightness/20
statout = sim.scan_fov(seq, xcoords, maxlocs=numberOfLocalizations, 
                       display=True, ax1=ax1v, clearfigure=False, tag="bg")
photonsbg = statout.phot[0]

In [ ]:
# background subtracted,
sim.background_estimated=sim.background*laserpower  # in general, the GT background is not known but needs to be calibrated 

sim.defineComponent("bg", "background", backgroundsubtractor, parameters=["background_estimated"])
seq = ["donut", "bg", "estdonut"]
psf_vec.zerooffset = 0
statout = sim.scan_fov(seq, xcoords, maxlocs=numberOfLocalizations, 
                       display=True, ax1=ax1v, clearfigure=False,
                       tag="bg est", title="estimator bias from background")

In [ ]:
sim.background_estimated = 0
sim.defineComponent("estimatorbg", "estimator", est_qDirectFitBg1D, parameters=[L], dim=0)
seq = ["donut", "estimatorbg"]
statout=sim.scan_fov(seq, xcoords, maxlocs=numberOfLocalizations, 
                     display=True, ax1=ax1v, clearfigure=False,
                     tag="bg fit par", title="estimator bias from background")

In [ ]:
if ax1v=="pos":
    ax.plot([0, L],[0, L],'k--')
ax.set_ylim([0, 65])
ax.set_xlim(right=L*0.75)

## Convergence of the iterative estimator

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
xcoords = np.arange(0,L+5,5)
iters = [1, 2, 3, 4, 5, 6, 7, 8, 10, 15, 20, 30, 50]

sim.background = 0
sim.background_estimated = 0
for it in iters:
    sim.defineComponent("estiter", "estimator", est_qLSQiter2D, parameters=[L,probecenter,it], dim=(0,1))
    seq = ["donut", "estiter"]
    statout=sim.scan_fov(seq, xcoords, maxlocs=numberOfLocalizations, 
                         display=True, ax1=ax1v, clearfigure=False, tag=f"iterations: {it}")
statout.phot
ax.plot([0, L],[0, L],'k--')
ax.set_ylim(bottom=0)
fig.tight_layout()